## This notebook is used for cleanup and formatting of NTSB data.

### Source: https://www.ntsb.gov/_layouts/ntsb.aviation/index.aspx
### File Format: Pipe Delimited '|'

Import the NTSB dataset

In [1]:
# import Pandas library
import pandas as pd 
filename = 'data/AviationData.txt' 
data = pd.read_csv(filename, delimiter='|')

In [2]:
# print shape of the dataframe
data.shape

(83386, 32)

In [3]:
# print first rows
data.head()

,Event Id,Investigation Type,Accident Number,Event Date,Location,Country,Latitude,Longitude,Airport Code,Airport Name,...,Air Carrier,Total Fatal Injuries,Total Serious Injuries,Total Minor Injuries,Total Uninjured,Weather Condition,Broad Phase of Flight,Report Status,Publication Date,
0,20190709X43210,Accident,GAA19CA397,07/08/2019,"Sterling City, TX",United States,,,,N/A,...,,,,,,,,Preliminary,07/10/2019,
1,20190707X23209,Accident,GAA19CA392,07/07/2019,"Westfield, IN",United States,40.048889,-86.157778,I72,Westfield,...,,,,,1,VMC,,Preliminary,07/10/2019,
2,20190706X10535,Accident,GAA19CA396,07/06/2019,"Anacortes, WA",United States,48.498611,-122.662500,74S,Anacortes,...,,,,,1,VMC,,Preliminary,07/10/2019,
3,20190708X21221,Accident,GAA19CA387,07/06/2019,"Seldovia, AK",United States,59.443889,-151.705000,SOV,Seldovia,...,,,,,5,VMC,,Preliminary,07/10/2019,
4,20190707X11324,Accident,CEN19LA214,07/06/2019,"Bay City, MI",United States,43.547500,-83.895000,3CM,James Clements Muni,...,,,1,,,VMC,MANEUVERING,Preliminary,07/11/2019,


Drop the last column

In [4]:
data = data[data.columns[:-1]]

In [5]:
# print out columns of the df
data.columns

Index(['Event Id ', ' Investigation Type ', ' Accident Number ',
       ' Event Date ', ' Location ', ' Country ', ' Latitude ', ' Longitude ',
       ' Airport Code ', ' Airport Name ', ' Injury Severity ',
       ' Aircraft Damage ', ' Aircraft Category ', ' Registration Number ',
       ' Make ', ' Model ', ' Amateur Built ', ' Number of Engines ',
       ' Engine Type ', ' FAR Description ', ' Schedule ',
       ' Purpose of Flight ', ' Air Carrier ', ' Total Fatal Injuries ',
       ' Total Serious Injuries ', ' Total Minor Injuries ',
       ' Total Uninjured ', ' Weather Condition ', ' Broad Phase of Flight ',
       ' Report Status ', ' Publication Date '],
      dtype='object')

Split out the Location field into 'City' and 'State'.

In [6]:
# pd.concat([data, data[' Location '].str.partition(',')[[0, 2]]], axis=1)
splitLocation = data[' Location '].str.split(", ", expand=True,)
splitLocation = splitLocation.drop(columns=[2, 3, 4], axis=1)

Add split fields to DataFrame

In [7]:
# data = pd.concat([data, splitLocation])
data["City"] = splitLocation[0]
data["State"] = splitLocation[1]

Rename the new columns.

In [8]:
 data = data.rename(index=str, columns={0: "City", 1: "State"})

Drop the Location column

In [9]:
data = data.drop(columns=[' Location '])

Change the dates format to %Y-%m-%d (to match what Splunk expects)

In [10]:
# convert Event Date
data[' Event Date '] = pd.to_datetime(data[' Event Date '])
data[' Event Date '] = data[' Event Date '].dt.strftime('%Y-%m-%d')

# convert Publication Date
data[' Publication Date '] = pd.to_datetime(data[' Publication Date '], errors='coerce')
data[' Publication Date '] = data[' Publication Date '].dt.strftime('%Y-%m-%d')

Save the updated DataFrame as CSV

In [11]:
data.to_csv("output/AviationData.csv", sep=',', index=False)